# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.


# Порядок выполнения 

Загрузка и подготовка данных.

Обучение и провека модели для каждого региона:
- Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- Обучите модель и сделайте предсказания на валидационной выборке.
- Сохраните предсказания и правильные ответы на валидационной выборке.
- Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- Проанализируйте результаты.

Подготовьтесь к расчёту прибыли:
- Все ключевые значения для расчётов сохраните в отдельных переменных.
- Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- Напишите выводы по этапу подготовки расчёта прибыли.

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберите скважины с максимальными значениями предсказаний.
- Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитайте прибыль для полученного объёма сырья.

Посчитайте риски и прибыль для каждого региона:
- Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

# Обозначения в таблицах

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).




## Загрузка и подготовка данных

### Импорт библиотек и модулей

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score,roc_curve, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display

### Чтение данных

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')# Регион 1
data_2 = pd.read_csv('/datasets/geo_data_1.csv')# Регион 2
data_3 = pd.read_csv('/datasets/geo_data_2.csv')# Регион 3

In [3]:
def info(data): # функция на просмотр общей информации к каждому датасету
    print('--------------------------------------------------------------------------')
    display(data.head())
    display(data.info())
    display(data.describe().T)
    print('Дубликатов:', data.duplicated().sum())
    print('--------------------------------------------------------------------------')
    print()

    
for data in [data_1, data_2, data_3]:
    info(data)
    
    

--------------------------------------------------------------------------


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


...

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


Дубликатов: 0
--------------------------------------------------------------------------



###  Разделение данных на выборки

In [4]:
target_1 = data_1['product']
target_2 = data_2['product']
target_3 = data_3['product']
# целевой признак
features_1 = data_1.drop(['product','id'], axis=1)
features_2 = data_2.drop(['product','id'], axis=1)
features_3 = data_3.drop(['product','id'], axis=1)
# переменные признаки('id' тоже удаляем, тк это незначимый признак, который может повлять на результыты работы модели)

In [5]:
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)
data_3_train, data_3_valid = train_test_split(data_3, test_size=0.25, random_state=12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.3, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.3, random_state=12345)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3, test_size=0.3, random_state=12345)

###  Вывод по разделу

- Мы имеем 3 таблицы по 10тыс строк и 4 столбца,  без пропусков и дубликатов
- Разделили данные на обучающии  и валидационные выборки в соотнощегии 75:25
- Избавились от столбца ID, тк не влияющее на результат значение



## Обучение и проверка модели

### Обучение модели

In [6]:
model = LinearRegression()

#### Регион 1

In [7]:
model.fit(features_train_1, target_train_1)
predictions_valid_1 = model.predict(features_valid_1)  
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5
mean_predictions_1 = predictions_valid_1.mean()
print(f' Регион 1: Средний запас предсказанного сырья: {mean_predictions_1}, RMSE = {rmse_1}')

 Регион 1: Средний запас предсказанного сырья: 92.56157209676121, RMSE = 37.617168381712055


#### Регион 2

In [8]:
model.fit(features_train_2, target_train_2)
predictions_valid_2 = model.predict(features_valid_2)  
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5
mean_predictions_2 = predictions_valid_2.mean()
print(f' Регион 2: Средний запас предсказанного сырья: {mean_predictions_2}, RMSE = {rmse_2}')

 Регион 2: Средний запас предсказанного сырья: 68.67768031635866, RMSE = 0.8883238359345488


#### Регион 3

In [9]:
model.fit(features_train_3, target_train_3)
predictions_valid_3 = model.predict(features_valid_3)  
rmse_3 = mean_squared_error(target_valid_3, predictions_valid_3)**0.5
mean_predictions_3 = predictions_valid_3.mean()
print(f' Регион 3: Средний запас предсказанного сырья: {mean_predictions_3}, RMSE = {rmse_3}')

 Регион 3: Средний запас предсказанного сырья: 94.93700245586417, RMSE = 40.0279480306151


### Вывод по разделу

- Самый низкий RMSE у второго региона, но самый маленький запас предсказонного сырья

## Подготовка к расчёту прибыли

In [10]:
budget = 10000000000
barrel_income = 450
one_unit_income = 450000
best_point = 200

In [11]:
break_even_point = (budget / one_unit_income) / best_point
print(f'Точка безубыточности {break_even_point}')

Точка безубыточности 111.11111111111111


### Вывод по разделу
- У всех регионов предсказазнный запас сырья меньше чем, точка безубыточности

## Расчёт прибыли и рисков 

### Функция расчета прибыли

In [12]:
def profit(target, predictions, count):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return (selected.sum() * one_unit_income) - budget

#### Регион 1

In [13]:
profit(target_valid_1, predictions_valid_1, best_point)

3360315346.487589

#### Регион 2

In [14]:
profit(target_valid_2, predictions_valid_2, best_point)

2415086696.681511

#### Регион 3

In [15]:
profit(target_valid_3, predictions_valid_3, best_point)

2624878110.5708427

### Bootstrap  процедура с 1000 итераций, которая высчитывает риски, среднее, и 95% доверительный интервал

In [16]:
def bootstrap(target, probabilities):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, best_point))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean() *100
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

    print('Средняя выручка:', mean)
    print('Риск убытков:', risk,'%')
    print('95% доверительный интервал:', confidence_interval)

#### Регион 1

In [17]:
bootstrap(target_valid_1.reset_index(drop=True), predictions_valid_1)

Средняя выручка: 385149309.7301583
Риск убытков: 8.3 %
95% доверительный интервал: (368286221.9423234, 402012397.5179933)


#### Регион 2

In [18]:
bootstrap(target_valid_2.reset_index(drop=True), predictions_valid_2)

Средняя выручка: 442183561.9700828
Риск убытков: 1.2 %
95% доверительный интервал: (429328617.59316987, 455038506.3469958)


#### Регион 3

In [19]:
bootstrap(target_valid_3.reset_index(drop=True), predictions_valid_3)

Средняя выручка: 391103435.0971428
Риск убытков: 7.9 %
95% доверительный интервал: (374507720.04497415, 407699150.1493115)


### Вывод по разделу
- Лучший регион для разработки скважин: Регион 2, т.к самый низкий риск убытков 1.2%, и высокая средняя предположительная выручка 442183561.9

## Вывод общий
- На подготовительном этапе мы:
  - Разделили данные на обучающии  и валидационные выборки в соотнощегии 75:25
  - Избавились от столбца ID, тк не влияющее на результат значение
- После обучения и проверки модели самый низкий RMSE был у модели Региона 2
- Написали функция для подсчета прибыли
- Написали функцию для процедуры Bootstrap, для подсчета рисков и средней выручки
- В итоге Лучший регион для разработки скважин: Регион 2, т.к самый низкий риск убытков 1.2%, и высокая средняя предположительная выручка 442183561.9
   